# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,greenville,35.6127,-77.3663,9.52,61,0,0.89,US,1700370747
1,1,isafjordur,66.0755,-23.1240,3.40,89,100,1.31,IS,1700371013
2,2,adamstown,-25.0660,-130.1015,21.80,87,100,9.46,PN,1700371013
3,3,hotan,37.0997,79.9269,9.48,15,81,1.30,CN,1700371000
4,4,ulladulla,-35.3500,150.4667,35.98,55,16,8.47,AU,1700370946


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.


In [3]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 700,
    size = "Humidity"
)

# Display the map plot
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness
preferred_cities_df = pd.DataFrame(city_data_df.loc[(city_data_df['Max Temp'] > 21) &
                                                     (city_data_df['Max Temp'] < 27) &
                                                     (city_data_df['Wind Speed'] < 4.5) &
                                                     (city_data_df['Cloudiness'] == 0)])

# Drop any rows with null values


# Display sample data
len(preferred_cities_df)

11

### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = preferred_cities_df[['City','Country','Lat','Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
187,santa luzia,BR,-19.7697,-43.8514,69,
239,taoudenni,ML,22.6783,-3.9836,22,
255,al wajh,SA,26.2455,36.4525,60,
321,dakhla,EH,23.6848,-15.9580,46,
332,saint-joseph,RE,-21.3667,55.6167,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set the parameters for the type of search
limit = 1
radius = 10000

# set up a parameters dictionary
params = {
    "categories":"accommodation.hotel",
    "limit":10,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
        


In [9]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, 'Lng']
    latitude = hotel_df.loc[index, 'Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filters"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_json = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
santa luzia - nearest hotel: Alcoboca
taoudenni - nearest hotel: فندق السايح
al wajh - nearest hotel: فندق رالين 2
dakhla - nearest hotel: Hotel El Massira
saint-joseph - nearest hotel: PALM Hotel & Spa
cabedelo - nearest hotel: No hotel found
antri - nearest hotel: Hotel Man Mandir
garachico - nearest hotel: Residencia El Drago
saint-philippe - nearest hotel: Le Baril
deoli - nearest hotel: Bundi Haveli
kharan - nearest hotel: SF Al-dawood Restaurant


,City,Country,Lat,Lng,Humidity,Hotel Name
187,santa luzia,BR,-19.7697,-43.8514,69,Alcoboca
239,taoudenni,ML,22.6783,-3.9836,22,فندق السايح
255,al wajh,SA,26.2455,36.4525,60,فندق رالين 2
321,dakhla,EH,23.6848,-15.9580,46,Hotel El Massira
332,saint-joseph,RE,-21.3667,55.6167,60,PALM Hotel & Spa
368,cabedelo,BR,-6.9811,-34.8339,83,No hotel found
380,antri,IN,26.0567,78.2144,36,Hotel Man Mandir
383,garachico,ES,28.7781,-17.7646,88,Residencia El Drago
481,saint-philippe,RE,-21.3585,55.7679,63,Le Baril
499,deoli,IN,25.7500,75.3833,33,Bundi Haveli


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
# Configure the map
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)